In [1]:
# Load in the packages
import numpy as np
import folktables
from folktables import ACSDataSource, ACSIncome

import os
import sys
import copy
sys.path.append('..')
import random
import FairCertModule
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models, transforms
from FullyConnected import FullyConnected
import pytorch_lightning as pl

# Set random seeds
SEED = 0
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

dataset = "Coverage"
if(dataset == "Employ"):
    TEST_EPSILON = 0.075
if(dataset == "Coverage"):
    TEST_EPSILON = 0.25
else:
    TEST_EPSILON = 0.05   


In [2]:
# Data loaders
import folk_utils
X_train, X_test, X_val, y_train, y_test, y_val, lp_epsilon, sr_epsilon = folk_utils.get_dataset(dataset)
f_epsilon = lp_epsilon

/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [3]:
# Load in the trained models

MODEL_STATE  = 'CA'
MODEL_YEAR   = '2015'
MODEL_WIDTH  = '256'
MODEL_METRIC = 'LP'

sgd_id = "SGD" 
pgd_id = "FAIR-PGD" 
ibp_id = "FAIR-IBP" 
glob_id = "FAIR-DRO"
ibpg_id = "FAIR-IBPG"

def load_model_from_id(model_id, dataset, width=MODEL_WIDTH):
    model = FullyConnected(hidden_lay=2, hidden_dim=256, dataset=dataset)
    ckpt = torch.load("%sModels/%s.ckpt"%(dataset, model_id))
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    checkpoint = torch.load("%sModels/%s.ckpt"%(dataset, model_id))
    model.load_state_dict(torch.load('%sModels/%s.pt'%(dataset, model_id)))
    return model

sgd_model = load_model_from_id(sgd_id, dataset)
pgd_model = load_model_from_id(pgd_id, dataset)
ibp_model = load_model_from_id(ibp_id, dataset)
glob_model = load_model_from_id(glob_id, dataset)
ibpg_model = load_model_from_id(ibpg_id, dataset)


In [4]:
from folk_utils import evaluate_accuracy, evaluate_delta_PGD, evaluate_delta_IBP

In [5]:
sgd_acc = evaluate_accuracy(sgd_model, X_test, y_test)
pgd_acc = evaluate_accuracy(pgd_model, X_test, y_test)
ibp_acc = evaluate_accuracy(ibp_model, X_test, y_test)
glob_acc = evaluate_accuracy(glob_model, X_test, y_test) 
ibpg_acc = evaluate_accuracy(ibpg_model, X_test, y_test)

In [6]:
print(sgd_acc)
print(pgd_acc)
print(ibp_acc)
print(glob_acc)
print(ibpg_acc)

tensor(0.7213)
tensor(0.7090)
tensor(0.6994)
tensor(0.6993)
tensor(0.6810)


In [7]:
# Evaluate the ood accuracy, lower, and upper bounds
f_epsilon = torch.Tensor(f_epsilon)
sgd_pgd = evaluate_delta_PGD(sgd_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2, iterations=25)
pgd_pgd = evaluate_delta_PGD(pgd_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2, iterations=25)
ibp_pgd = evaluate_delta_PGD(ibp_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2, iterations=25)
glob_pgd = evaluate_delta_PGD(glob_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2, iterations=25)
ibpg_pgd = evaluate_delta_PGD(ibpg_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2, iterations=25)

/Users/matthewwicker/Desktop/Development/CertificationOfDistributionalIF/Folktables/folk_utils.py:253: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = F.softmax(model(inp)).detach().numpy()
/Users/matthewwicker/Desktop/Development/CertificationOfDistributionalIF/Folktables/folk_utils.py:255: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_adv = F.softmax(model(x_adv)).detach().numpy()


In [8]:
print(sgd_pgd)
print(pgd_pgd)
print(ibp_pgd)
print(glob_pgd)
print(ibpg_pgd)

0.59620935
0.10999506
0.09371024
0.01292028
1.4992006e-10


In [9]:
# Evaluate the ood accuracy, lower, and upper bounds
f_epsilon = torch.Tensor(f_epsilon)
sgd_ibp = evaluate_delta_IBP(sgd_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2)
pgd_ibp = evaluate_delta_IBP(pgd_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2)
ibp_ibp = evaluate_delta_IBP(ibp_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2)
glob_ibp = evaluate_delta_IBP(glob_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2)
ibpg_ibp = evaluate_delta_IBP(ibpg_model, X_test, y_test, f_epsilon, TEST_EPSILON, 2)

In [10]:
print(sgd_ibp)
print(pgd_ibp)
print(ibp_ibp)
print(glob_ibp)
print(ibpg_ibp)

1.0
0.8488273
0.25593194
0.09324568
0.0387188


In [11]:
# Load in the geographical datasets 

def get_data_states(dataset, year):
    state_X_tests = []; state_y_tests = []
    state_lrs = []; state_srs = []
    year = str(year)
    states = ["AL", "AK", "AZ", "AR", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    for state in states:
        _, X_test, _, _, y_test, _, lp_epsilon, sr_epsilon = folk_utils.get_dataset(dataset, year, state)
        if(X_test.shape[-1] != X_val.shape[-1]):
            continue
        state_X_tests.append(X_test)
        state_y_tests.append(y_test)
        state_lrs.append(lp_epsilon); state_srs.append(sr_epsilon)
    return state_X_tests, state_y_tests

X_test_states, y_test_states = get_data_states(dataset, '2015')

/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[N

/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[N

/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[N

/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[N

In [12]:
sgd_ibps = []
pgd_ibps = []
ibp_ibps = []
glob_ibps = []
ibpg_ibps = []
for X, y in zip(X_test_states, y_test_states):
    X = torch.Tensor(X)
    y = torch.Tensor(y).type(torch.LongTensor)
    sa = evaluate_delta_IBP(sgd_model, X, y, f_epsilon, TEST_EPSILON, 2)
    sgd_ibps.append(sa)
    
    pa = evaluate_delta_IBP(pgd_model, X, y, f_epsilon, TEST_EPSILON, 2)
    pgd_ibps.append(pa)
    
    ia = evaluate_delta_IBP(ibp_model, X, y, f_epsilon, TEST_EPSILON, 2)
    ibp_ibps.append(ia)
    
    ga = evaluate_delta_IBP(glob_model, X, y, f_epsilon, TEST_EPSILON, 2)
    glob_ibps.append(ga)
    
    da = evaluate_delta_IBP(ibpg_model, X, y, f_epsilon, TEST_EPSILON, 2)
    ibpg_ibps.append(da)

In [13]:
print(max(sgd_ibps))
print(max(pgd_ibps))
print(max(ibp_ibps))
print(max(glob_ibps))
print(max(ibpg_ibps))


1.0
0.86506057
0.253249
0.12892541
0.040500473


In [18]:
# Compute distributional robustness upper and lower bounds
from folk_utils import compute_DIF_certification, compute_DIF_falsification
from FairCertModule import *
from tqdm import trange

TEST_EPSILON = 0.25
def compute_cert(model):
    dro1, ibp_trend, vg = compute_DIF_certification(model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, lr = 1, N=1000, iters=200, rettrend=True) 
    dro2, ibp_trend, vg = compute_DIF_certification(model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, lr = 0.1, N=1000, iters=1000, rettrend=True) 
    return max(dro1, dro2)

#sgd_dro, ibp_trend, vg = compute_DIF_certification(sgd_model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, lr = 1, N=1000, iters=1000, rettrend=True) 
#pgd_dro, ibp_trend, vg = compute_DIF_certification(pgd_model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, N=1000, iters=1000, rettrend=True)   
#ibp_dro, ibp_trend, vg = compute_DIF_certification(ibp_model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, N=1000, iters=1000, rettrend=True)  
#glob_dro, glob_trend, vg = compute_DIF_certification(glob_model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, N=1000, iters=1000, rettrend=True)
#ibpg_dro, glob_trend, vg = compute_DIF_certification(ibpg_model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, N=1000, iters=1000, rettrend=True)

sgd_dro = compute_cert(sgd_model)
pgd_dro = compute_cert(pgd_model)
ibp_dro = compute_cert(ibp_model)
glob_dro = compute_cert(glob_model)
ibpg_dro = compute_cert(ibpg_model)

100%|███████████████████████████████████████| 1000/1000 [00:16<00:00, 60.06it/s]


In [19]:
print((sgd_dro))
print((pgd_dro))
print((ibp_dro))
print((glob_dro))
print((ibpg_dro))


tensor(1., grad_fn=<MeanBackward0>)
tensor(0.8891, grad_fn=<MeanBackward0>)
tensor(0.2886, grad_fn=<MeanBackward0>)
tensor(0.1360, grad_fn=<MeanBackward0>)
tensor(0.0713, grad_fn=<MeanBackward0>)


In [20]:
dro1, ibp_trend, vg = compute_DIF_certification(ibp_model, f_epsilon, TEST_EPSILON, 0.025, X_test, y_test, lr = 10, N=1000, iters=1000, rettrend=True)
print(dro1)


100%|███████████████████████████████████████| 1000/1000 [00:16<00:00, 59.42it/s]

tensor(0.2754, grad_fn=<MeanBackward0>)
